In [1]:
import numpy as np
import random
from scipy.stats import norm
from selectinf.nbd_lasso import nbd_lasso
from selectinf.Utils.discrete_family import discrete_family
from selectinf.Tests.instance import GGM_instance
from selectinf.nbd_helpers import is_sym

from selectinf.Tests.nbd_naive_and_ds import *

In [256]:
def GGM_instance(n=100, p=100, max_edges=10):
    def generate_vertices(p):
        vertices = np.random.uniform(size=(p,2))
        return vertices
    def connecting_prob(v1,v2,p):
        # Euclidean distance of v1, v2
        d = np.linalg.norm(v1-v2)
        # calculating connecting probability
        prob = norm.pdf(d/np.sqrt(p))
        return prob
    def remove_edges(p, adj, max_edges):
        idx = list(range(p))
        np.random.shuffle(idx)

        for i in idx:
            if np.all(np.sum(adj, axis=1) <= (max_edges+1)):
                break
            # Indices of nodes connected to v_i
            nonzero_i = list(np.nonzero(adj[i])[0])
            n_edges = len(nonzero_i)

            # Delete some edges if there are redundancies
            if n_edges > (max_edges+1):
                nonzero_i.remove(i)
                removed_idx_i = random.sample(nonzero_i,n_edges-max_edges)
                # Remove other edges
                adj[i,removed_idx_i] = 0
                adj[removed_idx_i,i] = 0

        return adj

    vertices = generate_vertices(p)

    adj_mat = np.eye(p)

    for i in range(p):
        for j in range(i+1,p):
            v_i = vertices[i]
            v_j = vertices[j]
            adj_mat[i,j] = np.random.binomial(n=1,
                                              p=connecting_prob(v1=v_i,
                                                                v2=v_j,
                                                                p=p))

    # symmetrize
    adj_mat = adj_mat + adj_mat.T - np.eye(p)

    # remove redundant edges
    adj_mat = remove_edges(p, adj_mat, max_edges)

    # maximal off-diag value to guarantee diagonal dominance
    max_off_diag = 1/max_edges
    max_off_diag = max_off_diag*0.9

    # generate a PD precision
    precision = np.random.uniform(low=-max_off_diag,high=max_off_diag,
                                  size=(p,p))
    # symmetrize precision
    precision = np.tril(precision)
    precision = precision + precision.T
    # sparsify precision based on adjacency matrix
    precision = precision * adj_mat
    np.fill_diagonal(precision, 1)
    cov = np.linalg.inv(precision)

    # standardize the covariance
    cov = cov/np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    precision = np.linalg.inv(cov)

    X = np.random.multivariate_normal(mean=np.zeros(p),
                                      cov=cov, size=n)

    return precision, cov, X

In [259]:
prec,cov,X = GGM_instance(n=3000, p=50, max_edges=3)

In [260]:
weights_const=1.75
ridge_const=1.
randomizer_scale=5.

nbd_instance = nbd_lasso.gaussian(X, n_scaled=False, weights_const=weights_const,
                                      ridge_terms=ridge_const, randomizer_scale=randomizer_scale)
active_signs_random = nbd_instance.fit()
nonzero = nbd_instance.nonzero

# Construct intervals
if nonzero.sum() > 0:
    # Intervals returned is in its original (unscaled) order
    intervals = nbd_instance.inference(parallel=False)

Inference for 2 , 49
Inference for 3 , 13
Inference for 6 , 40
Inference for 7 , 13
Inference for 8 , 40
Inference for 9 , 19
Inference for 10 , 47
Inference for 11 , 12
Inference for 12 , 22
Inference for 12 , 28
Inference for 15 , 37
Inference for 15 , 45
Inference for 18 , 27
Inference for 18 , 28
Inference for 24 , 31
Inference for 24 , 42
Inference for 25 , 45
Inference for 31 , 47
Inference for 32 , 37
Inference for 34 , 39
Inference for 35 , 47
Inference for 37 , 44


In [261]:
n=3000
p=50
coverage = get_coverage(nonzero, intervals, prec, n=3000, p=50, scale=False)
interval_len = 0
nonzero_count = 0  # nonzero_count is essentially upper-triangular
for i in range(p):
    for j in range(i+1,p):
        if nonzero[i,j]:
            interval = intervals[i,j,:]
            interval_len = interval_len + (interval[1] - interval[0])
            nonzero_count = nonzero_count + 1
avg_len = interval_len / nonzero_count
cov_rate = coverage.sum() / nonzero_count
F1_approx = calculate_F1_score_graph(prec, selection=nonzero)

In [262]:
cov_rate

0.8636363636363636

In [263]:
avg_len

0.06587257265832323

In [264]:
F1_approx

0.21390374331550804

In [220]:
intervals[29,42,:]

array([0., 0.])

In [15]:
prec[29,42]

-1.1994980566305345e-19

In [78]:
def print_nonzero_intervals(nonzero, intervals, prec, X):
    # Intervals, prec, X are all in their original scale
    n, p = X.shape
    S = X.T @ X / n

    for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    print("(",i,",",j,")", "selected")
                    print("Theta", "(",i,",",j,")", "interval:", intervals[i,j,:])
                    print("Theta", "(",i,",",j,")", prec[i,j])
                    print("S/n", "(",i,",",j,")", S[i,j])
print_nonzero_intervals(nonzero, intervals, prec, X)

( 0 , 40 ) selected
Theta ( 0 , 40 ) interval: [-0.27950555 -0.19836488]
Theta ( 0 , 40 ) -0.2418773674317705
S/n ( 0 , 40 ) 0.27323961786239637
( 8 , 12 ) selected
Theta ( 8 , 12 ) interval: [-0.26610986 -0.1814441 ]
Theta ( 8 , 12 ) -0.2367285672028064
S/n ( 8 , 12 ) 0.22377826508321755
( 10 , 16 ) selected
Theta ( 10 , 16 ) interval: [-0.29178851 -0.21623954]
Theta ( 10 , 16 ) -0.23181282989712362
S/n ( 10 , 16 ) 0.27547688097100087
( 12 , 35 ) selected
Theta ( 12 , 35 ) interval: [-0.26256417 -0.18403046]
Theta ( 12 , 35 ) -0.2361518908837335
S/n ( 12 , 35 ) 0.23539716390157203
( 15 , 38 ) selected
Theta ( 15 , 38 ) interval: [-0.26406449 -0.18878303]
Theta ( 15 , 38 ) -0.224035239893576
S/n ( 15 , 38 ) 0.2396799828011466
( 19 , 30 ) selected
Theta ( 19 , 30 ) interval: [-0.29206633 -0.20795965]
Theta ( 19 , 30 ) -0.24833629755997633
S/n ( 19 , 30 ) 0.22928719203778375
( 19 , 31 ) selected
Theta ( 19 , 31 ) interval: [-0.27103762 -0.18703016]
Theta ( 19 , 31 ) -0.26251670201630967


## Random Edges Instance

In [6]:
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

In [7]:
def GGM_random_instances(n=200, p=50, theta=-0.2):

    # Guarantee same sparsity level as in Friedman et al.:
    # https://www.asc.ohio-state.edu/statistics/statgen/joul_aut2015/2010-Friedman-Hastie-Tibshirani.pdf
    prob = 0.4 / (np.abs(theta)*p)

    invertible = False

    # Generate invertible precision
    while not invertible:
        prec = np.eye(p)

        # Randomly selecting edges
        for i in range(p):
            for j in range(i + 1, p):
                prec[i, j] = theta * np.random.binomial(n=1, p=prob)

        # symmetrize
        prec = prec + prec.T - np.eye(p)

        invertible = is_invertible(prec)

    cov = np.linalg.inv(prec)
    # standardize the covariance
    cov = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    prec = np.linalg.inv(cov)

    X = np.random.multivariate_normal(mean=np.zeros(p),
                                      cov=cov, size=n)

    return prec, cov, X

In [221]:
prec,cov,X = GGM_random_instances(n=3000, p=50, theta=-0.2)

In [232]:
weights_const=1.75
ridge_const=1.
randomizer_scale=5.

nbd_instance = nbd_lasso.gaussian(X, n_scaled=False, weights_const=weights_const,
                                      ridge_terms=ridge_const, randomizer_scale=randomizer_scale)
active_signs_random = nbd_instance.fit()
nonzero = nbd_instance.nonzero

# Construct intervals
if nonzero.sum() > 0:
    # Intervals returned is in its original (unscaled) order
    intervals = nbd_instance.inference(parallel=False)

Inference for 1 , 16
Inference for 1 , 17
Inference for 1 , 20
Inference for 1 , 26
Inference for 1 , 43
Inference for 2 , 29
Inference for 2 , 37
Inference for 2 , 41
Inference for 3 , 13
Inference for 3 , 14
Inference for 4 , 5
Inference for 4 , 38
Inference for 4 , 41
Inference for 5 , 16
Inference for 5 , 17
Inference for 6 , 29
Inference for 6 , 36
Inference for 7 , 44
Inference for 8 , 20
Inference for 8 , 32
Inference for 9 , 14
Inference for 10 , 26
Inference for 10 , 35
Inference for 10 , 37
Inference for 11 , 37
Inference for 13 , 19
Inference for 13 , 23
Inference for 13 , 47
Inference for 14 , 39
Inference for 14 , 40
Inference for 14 , 48
Inference for 14 , 49
Inference for 15 , 26
Inference for 15 , 30
Inference for 15 , 42
Inference for 16 , 17
Inference for 18 , 40
Inference for 19 , 27
Inference for 20 , 35
Inference for 20 , 44
Inference for 23 , 27
Inference for 23 , 31
Inference for 27 , 31
Inference for 27 , 32
Inference for 27 , 35
Inference for 28 , 42
Inference 

In [233]:
n=3000
p=50
coverage = get_coverage(nonzero, intervals, prec, n=3000, p=50, scale=False)
interval_len = 0
nonzero_count = 0  # nonzero_count is essentially upper-triangular
for i in range(p):
    for j in range(i+1,p):
        if nonzero[i,j]:
            interval = intervals[i,j,:]
            interval_len = interval_len + (interval[1] - interval[0])
            nonzero_count = nonzero_count + 1
avg_len = interval_len / nonzero_count
cov_rate = coverage.sum() / nonzero_count
F1_approx = calculate_F1_score_graph(prec, selection=nonzero)

In [234]:
cov_rate

0.8653846153846154

In [235]:
avg_len

0.07477661482446482

In [236]:
F1_approx

0.10321221695629278

In [220]:
intervals[29,42,:]

array([0., 0.])

In [15]:
prec[29,42]

-1.1994980566305345e-19

In [78]:
def print_nonzero_intervals(nonzero, intervals, prec, X):
    # Intervals, prec, X are all in their original scale
    n, p = X.shape
    S = X.T @ X / n

    for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    print("(",i,",",j,")", "selected")
                    print("Theta", "(",i,",",j,")", "interval:", intervals[i,j,:])
                    print("Theta", "(",i,",",j,")", prec[i,j])
                    print("S/n", "(",i,",",j,")", S[i,j])
print_nonzero_intervals(nonzero, intervals, prec, X)

( 0 , 40 ) selected
Theta ( 0 , 40 ) interval: [-0.27950555 -0.19836488]
Theta ( 0 , 40 ) -0.2418773674317705
S/n ( 0 , 40 ) 0.27323961786239637
( 8 , 12 ) selected
Theta ( 8 , 12 ) interval: [-0.26610986 -0.1814441 ]
Theta ( 8 , 12 ) -0.2367285672028064
S/n ( 8 , 12 ) 0.22377826508321755
( 10 , 16 ) selected
Theta ( 10 , 16 ) interval: [-0.29178851 -0.21623954]
Theta ( 10 , 16 ) -0.23181282989712362
S/n ( 10 , 16 ) 0.27547688097100087
( 12 , 35 ) selected
Theta ( 12 , 35 ) interval: [-0.26256417 -0.18403046]
Theta ( 12 , 35 ) -0.2361518908837335
S/n ( 12 , 35 ) 0.23539716390157203
( 15 , 38 ) selected
Theta ( 15 , 38 ) interval: [-0.26406449 -0.18878303]
Theta ( 15 , 38 ) -0.224035239893576
S/n ( 15 , 38 ) 0.2396799828011466
( 19 , 30 ) selected
Theta ( 19 , 30 ) interval: [-0.29206633 -0.20795965]
Theta ( 19 , 30 ) -0.24833629755997633
S/n ( 19 , 30 ) 0.22928719203778375
( 19 , 31 ) selected
Theta ( 19 , 31 ) interval: [-0.27103762 -0.18703016]
Theta ( 19 , 31 ) -0.26251670201630967


In [80]:
sum(sum(np.abs(prec) > 1e-10))/2

82.0

In [81]:
nonzero_count

12

## Hub Instance

In [58]:
def GGM_hub_instances(n=200, p=50, K=10, theta=-0.175):
    group_size = int(p / K)

    invertible = False
    while not invertible:
        prec = np.eye(p)
        for k in range(K):
            group_k = range(k * group_size, (k + 1) * group_size)
            hub = random.sample(list(group_k), 1)[0]
            for i in group_k:
                # fix column at hub, iterate over all rows in the group
                if i != hub:
                    prec[i, hub] = theta

        # symmetrize
        prec = prec + prec.T - np.eye(p)

        invertible = is_invertible(prec)

    cov = np.linalg.inv(prec)
    # standardize the covariance
    cov = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    prec = np.linalg.inv(cov)

    X = np.random.multivariate_normal(mean=np.zeros(p),
                                      cov=cov, size=n)

    return prec, cov, X

In [237]:
prec, cov, X = GGM_hub_instances(n=3000, p=50, K=10, theta=-0.2)

In [238]:
weights_const=2
ridge_const=1.
randomizer_scale=5

nbd_instance = nbd_lasso.gaussian(X, n_scaled=False, weights_const=weights_const,
                                      ridge_terms=ridge_const, randomizer_scale=randomizer_scale)
active_signs_random = nbd_instance.fit()
nonzero = nbd_instance.nonzero

# Construct intervals
if nonzero.sum() > 0:
    # Intervals returned is in its original (unscaled) order
    intervals = nbd_instance.inference(parallel=False)

Inference for 0 , 4
Inference for 5 , 6
Inference for 5 , 7
Inference for 11 , 14
Inference for 14 , 22
Inference for 15 , 18
Inference for 16 , 18
Inference for 25 , 27
Inference for 27 , 28
Inference for 28 , 47
Inference for 30 , 34
Inference for 32 , 34
Inference for 33 , 34
Inference for 35 , 36
Inference for 35 , 38
Inference for 41 , 43
Inference for 42 , 43
Inference for 45 , 48


In [239]:
n=3000
p=50
coverage = get_coverage(nonzero, intervals, prec, n=3000, p=50, scale=False)
interval_len = 0
nonzero_count = 0  # nonzero_count is essentially upper-triangular
for i in range(p):
    for j in range(i+1,p):
        if nonzero[i,j]:
            interval = intervals[i,j,:]
            interval_len = interval_len + (interval[1] - interval[0])
            nonzero_count = nonzero_count + 1
avg_len = interval_len / nonzero_count
cov_rate = coverage.sum() / nonzero_count
F1_approx = calculate_F1_score_graph(prec, selection=nonzero)

In [240]:
cov_rate

0.8888888888888888

In [241]:
avg_len

0.07009583081064891

In [242]:
F1_approx

0.2922374429223744

In [243]:
def print_nonzero_intervals(nonzero, intervals, prec, X):
    # Intervals, prec, X are all in their original scale
    n, p = X.shape
    S = X.T @ X / n

    for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    print("(",i,",",j,")", "selected")
                    print("Theta", "(",i,",",j,")", "interval:", intervals[i,j,:])
                    print("Theta", "(",i,",",j,")", prec[i,j])
                    print("S/n", "(",i,",",j,")", S[i,j])
print_nonzero_intervals(nonzero, intervals, prec, X)

( 0 , 4 ) selected
Theta ( 0 , 4 ) interval: [-0.276  -0.2047]
Theta ( 0 , 4 ) -0.22335313142016328
S/n ( 0 , 4 ) 0.23516095438525988
( 5 , 6 ) selected
Theta ( 5 , 6 ) interval: [-0.2444 -0.1729]
Theta ( 5 , 6 ) -0.22335313142016336
S/n ( 5 , 6 ) 0.1902661616819056
( 5 , 7 ) selected
Theta ( 5 , 7 ) interval: [-0.2444 -0.1715]
Theta ( 5 , 7 ) -0.22335313142016333
S/n ( 5 , 7 ) 0.1958561909890243
( 11 , 14 ) selected
Theta ( 11 , 14 ) interval: [-0.2704 -0.1993]
Theta ( 11 , 14 ) -0.22335313142016333
S/n ( 11 , 14 ) 0.22905076445323674
( 14 , 22 ) selected
Theta ( 14 , 22 ) interval: [-0.048   0.0187]
Theta ( 14 , 22 ) 0.0
S/n ( 14 , 22 ) 0.00806671789840488
( 15 , 18 ) selected
Theta ( 15 , 18 ) interval: [-0.2703 -0.2006]
Theta ( 15 , 18 ) -0.2233531314201633
S/n ( 15 , 18 ) 0.23692413504974938
( 16 , 18 ) selected
Theta ( 16 , 18 ) interval: [-0.2882 -0.2156]
Theta ( 16 , 18 ) -0.22335313142016333
S/n ( 16 , 18 ) 0.2335621826327831
( 25 , 27 ) selected
Theta ( 25 , 27 ) interval: [-

In [264]:
sum(sum(np.abs(prec) > 1e-10))/2

65.0

In [266]:
sum(sum(nonzero))/2

68.0

## Clique Instance

In [87]:
def GGM_clique_instances(n=200, p=400, K=20, group_size=7, theta=-0.175):
    # Partition [p] into p/K (big_group_size) disjoint sets,
    # then choose a fixed-size subset of each disjoint set

    assert K * group_size < p
    big_group_size = int(p/K)

    invertible = False
    while not invertible:
        prec = np.eye(p)
        for k in range(K):
            group_k = range(k * big_group_size, (k + 1) * big_group_size)
            variables_k = np.random.choice(group_k,
                                           size=group_size, replace=False)
            for i in variables_k:
                for j in variables_k:
                    # Set theta_ij = theta
                    if i != j:
                        prec[i, j] = theta

        invertible = is_invertible(prec)

    cov = np.linalg.inv(prec)
    # standardize the covariance
    cov = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    prec = np.linalg.inv(cov)

    X = np.random.multivariate_normal(mean=np.zeros(p),
                                      cov=cov, size=n)

    return prec, cov, X

In [250]:
prec, cov, X = GGM_clique_instances(n=3000, p=50, K=10, group_size=4, theta=-0.2)

In [251]:
weights_const=2.
ridge_const=1.
randomizer_scale=5.

nbd_instance = nbd_lasso.gaussian(X, n_scaled=False, weights_const=weights_const,
                                      ridge_terms=ridge_const, randomizer_scale=randomizer_scale)
active_signs_random = nbd_instance.fit()
nonzero = nbd_instance.nonzero

# Construct intervals
if nonzero.sum() > 0:
    # Intervals returned is in its original (unscaled) order
    intervals = nbd_instance.inference(parallel=False)

Inference for 0 , 1
Inference for 1 , 2
Inference for 1 , 4
Inference for 5 , 9
Inference for 6 , 9
Inference for 8 , 9
Inference for 10 , 11
Inference for 10 , 12
Inference for 12 , 14
Inference for 16 , 17
Inference for 16 , 18
Inference for 16 , 19
Inference for 17 , 18
Inference for 17 , 19
Inference for 18 , 19
Inference for 20 , 22
Inference for 20 , 23
Inference for 20 , 24
Inference for 23 , 24
Inference for 25 , 27
Inference for 26 , 27
Inference for 26 , 29
Inference for 27 , 29
Inference for 31 , 32
Inference for 31 , 33
Inference for 31 , 34
Inference for 32 , 33
Inference for 32 , 34
Inference for 35 , 36
Inference for 35 , 38
Inference for 35 , 39
Inference for 36 , 37
Inference for 37 , 38
Inference for 40 , 41
Inference for 40 , 44
Inference for 41 , 43
Inference for 41 , 44
Inference for 41 , 48
Inference for 43 , 44
Inference for 45 , 46
Inference for 45 , 47
Inference for 45 , 49
Inference for 46 , 47
Inference for 46 , 49


In [252]:
n=3000
p=50
coverage = get_coverage(nonzero, intervals, prec, n=3000, p=50, scale=False)
interval_len = 0
nonzero_count = 0  # nonzero_count is essentially upper-triangular
for i in range(p):
    for j in range(i+1,p):
        if nonzero[i,j]:
            interval = intervals[i,j,:]
            interval_len = interval_len + (interval[1] - interval[0])
            nonzero_count = nonzero_count + 1
avg_len = interval_len / nonzero_count
cov_rate = coverage.sum() / nonzero_count
F1_approx = calculate_F1_score_graph(prec, selection=nonzero)

In [253]:
cov_rate

0.9318181818181818

In [254]:
avg_len

0.07836128656526237

In [255]:
F1_approx

0.8076923076923077

In [100]:
def print_nonzero_intervals(nonzero, intervals, prec, X):
    # Intervals, prec, X are all in their original scale
    n, p = X.shape
    S = X.T @ X / n

    for i in range(p):
            for j in range(i+1,p):
                if nonzero[i,j]:
                    print("(",i,",",j,")", "selected")
                    print("Theta", "(",i,",",j,")", "interval:", intervals[i,j,:])
                    print("Theta", "(",i,",",j,")", prec[i,j])
                    print("S/n", "(",i,",",j,")", S[i,j])
print_nonzero_intervals(nonzero, intervals, prec, X)

( 1 , 2 ) selected
Theta ( 1 , 2 ) interval: [-0.3019 -0.2141]
Theta ( 1 , 2 ) -0.25000000000000006
S/n ( 1 , 2 ) 0.3335624118532152
( 1 , 3 ) selected
Theta ( 1 , 3 ) interval: [-0.2584 -0.1708]
Theta ( 1 , 3 ) -0.2500000000000001
S/n ( 1 , 3 ) 0.3058251012754842
( 1 , 4 ) selected
Theta ( 1 , 4 ) interval: [-0.3223 -0.2354]
Theta ( 1 , 4 ) -0.2500000000000001
S/n ( 1 , 4 ) 0.34856373433688903
( 2 , 3 ) selected
Theta ( 2 , 3 ) interval: [-0.3018 -0.2141]
Theta ( 2 , 3 ) -0.25000000000000006
S/n ( 2 , 3 ) 0.32573139452357397
( 2 , 4 ) selected
Theta ( 2 , 4 ) interval: [-0.3156 -0.2285]
Theta ( 2 , 4 ) -0.25000000000000006
S/n ( 2 , 4 ) 0.34589468163157866
( 3 , 4 ) selected
Theta ( 3 , 4 ) interval: [-0.2665 -0.1801]
Theta ( 3 , 4 ) -0.25000000000000006
S/n ( 3 , 4 ) 0.32352521302826587
( 5 , 6 ) selected
Theta ( 5 , 6 ) interval: [-0.3369 -0.2488]
Theta ( 5 , 6 ) -0.25000000000000006
S/n ( 5 , 6 ) 0.34567518226625954
( 5 , 7 ) selected
Theta ( 5 , 7 ) interval: [-0.2698 -0.1833]
The

In [101]:
sum(sum(np.abs(prec) > 1e-10))/2

85.0

In [102]:
sum(sum(np.abs(nonzero) > 1e-10))/2

52.0